# Deep cortical machine learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.7618828170398175 -2.7965065300181307
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
# Data of the model
def model_input(state_size):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    return states, actions, targetQs, reward

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [16]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward):
    # G
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions[:-1] * targetQs[1:])
    
    # D
    Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_lossR = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits,
                                                                     labels=reward*tf.ones_like(Qs_logits)))
    d_lossQ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits[:-1],
                                                                     labels=tf.reshape(tf.nn.sigmoid(targetQs[1:]),
                                                                                       shape=[-1, 1])))
    d_loss = d_lossR #+ d_lossQ

    return actions_logits, Qs_logits, g_loss, d_loss, d_lossR, d_lossQ

In [17]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss, self.d_lossR, self.d_lossQ = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, # model input
            targetQs=self.targetQs, reward=self.reward) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [19]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [20]:
# Training parameters
# Network parameters
state_size = 4               # number of units for the input state/observation -- simulation
action_size = 2              # number of units for the output actions -- simulation
hidden_size = 64             # number of units in each Q-network hidden layer -- simulation
learning_rate = 0.001          # learning rate for adam

In [21]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [22]:
# import gym

# ## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# # env = gym.make('MountainCar-v0')
# # env = gym.make('Pendulum-v0')
# # env = gym.make('Blackjack-v0')
# # env = gym.make('FrozenLake-v0')
# # env = gym.make('AirRaid-ram-v0')
# # env = gym.make('AirRaid-v0')
# # env = gym.make('BipedalWalker-v2')
# # env = gym.make('Copy-v0')
# # env = gym.make('CarRacing-v0')
# # env = gym.make('Ant-v2') #mujoco
# # env = gym.make('FetchPickAndPlace-v1') # mujoco required!

In [ ]:
from collections import deque
episodes_total_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []
d_lossR_list, d_lossQ_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(111111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state

        # Training steps/batches
        while True:
            action_logits, Q_logits = sess.run(fetches=[model.actions_logits, model.Qs_logits], 
                                               feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            batch.append([state, action, Q_logits])
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done is True: # episode ended success/failure
                episodes_total_reward.append(total_reward) # stopping criteria
                rate = total_reward/ 500 # success is 500 points
                break

        # Training using batches
        #batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        targetQs = np.array([each[2] for each in batch])
        g_loss, d_loss, d_lossR, d_lossQ, _, _ = sess.run([model.g_loss, model.d_loss, 
                                                           model.d_lossR, model.d_lossQ, 
                                                           model.g_opt, model.d_opt],
                                                          feed_dict = {model.states: states, 
                                                                       model.actions: actions,
                                                                       model.reward: rate,
                                                                       model.targetQs: targetQs.reshape([-1])})
        # Average 100 episode total reward
        # Print out
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episodes_total_reward)),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss),
              'dlossR:{:.4f}'.format(d_lossR),
              'dlossQ:{:.4f}'.format(d_lossQ),
)
        # Ploting out
        rewards_list.append([ep, np.mean(episodes_total_reward)])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        d_lossR_list.append([ep, d_lossR])
        d_lossQ_list.append([ep, d_lossQ])
        # Break episode/epoch loop
        if np.mean(episodes_total_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model2.ckpt')

Episode:0 meanR:46.0000 gloss:-0.0105 dloss:0.6879 dlossR:0.6879 dlossQ:0.6925
Episode:1 meanR:138.5000 gloss:0.0243 dloss:0.6955 dlossR:0.6955 dlossQ:0.6923
Episode:2 meanR:99.3333 gloss:0.0103 dloss:0.6993 dlossR:0.6993 dlossQ:0.6931
Episode:3 meanR:78.0000 gloss:0.0492 dloss:0.7240 dlossR:0.7240 dlossQ:0.6924
Episode:4 meanR:64.4000 gloss:0.0422 dloss:0.7186 dlossR:0.7186 dlossQ:0.6922
Episode:5 meanR:55.1667 gloss:0.0073 dloss:0.6942 dlossR:0.6942 dlossQ:0.6930
Episode:6 meanR:48.5714 gloss:-0.0253 dloss:0.6712 dlossR:0.6712 dlossQ:0.6929
Episode:7 meanR:43.7500 gloss:-0.0599 dloss:0.6476 dlossR:0.6476 dlossQ:0.6920
Episode:8 meanR:40.0000 gloss:-0.0982 dloss:0.6203 dlossR:0.6203 dlossQ:0.6898
Episode:9 meanR:36.8000 gloss:-0.1347 dloss:0.5985 dlossR:0.5985 dlossQ:0.6872
Episode:10 meanR:34.3636 gloss:-0.1867 dloss:0.5703 dlossR:0.5703 dlossQ:0.6819
Episode:11 meanR:32.1667 gloss:-0.2226 dloss:0.5528 dlossR:0.5528 dlossQ:0.6785
Episode:12 meanR:30.4615 gloss:-0.2882 dloss:0.5231 dl

Episode:105 meanR:30.8700 gloss:-1.7023 dloss:0.6261 dlossR:0.6261 dlossQ:0.2744
Episode:106 meanR:32.4800 gloss:-1.7848 dloss:0.9572 dlossR:0.9572 dlossQ:0.2528
Episode:107 meanR:32.9900 gloss:-1.7318 dloss:0.3853 dlossR:0.3853 dlossQ:0.2653
Episode:108 meanR:33.9800 gloss:-1.7852 dloss:0.6386 dlossR:0.6386 dlossQ:0.2533
Episode:109 meanR:34.4800 gloss:-1.7358 dloss:0.3701 dlossR:0.3701 dlossQ:0.2650
Episode:110 meanR:34.6800 gloss:-1.7164 dloss:0.2292 dlossR:0.2292 dlossQ:0.2704
Episode:111 meanR:34.8000 gloss:-1.7736 dloss:0.1768 dlossR:0.1768 dlossQ:0.2561
Episode:112 meanR:34.8900 gloss:-1.8613 dloss:0.1682 dlossR:0.1682 dlossQ:0.2360
Episode:113 meanR:34.8500 gloss:-1.8814 dloss:0.1575 dlossR:0.1575 dlossQ:0.2318
Episode:114 meanR:34.7000 gloss:-1.9276 dloss:0.1669 dlossR:0.1669 dlossQ:0.2216
Episode:115 meanR:34.3400 gloss:-1.9328 dloss:0.1393 dlossR:0.1393 dlossQ:0.2207
Episode:116 meanR:33.5900 gloss:-2.1017 dloss:0.1177 dlossR:0.1177 dlossQ:0.1883
Episode:117 meanR:31.3300 gl

Episode:209 meanR:47.8800 gloss:-1.9607 dloss:0.1229 dlossR:0.1229 dlossQ:0.2201
Episode:210 meanR:47.6700 gloss:-1.9431 dloss:0.1187 dlossR:0.1187 dlossQ:0.2240
Episode:211 meanR:47.6100 gloss:-1.8661 dloss:0.1468 dlossR:0.1468 dlossQ:0.2410
Episode:212 meanR:47.6300 gloss:-1.5528 dloss:0.1978 dlossR:0.1978 dlossQ:0.3193
Episode:213 meanR:47.7000 gloss:-1.7077 dloss:0.2072 dlossR:0.2072 dlossQ:0.2843
Episode:214 meanR:48.3600 gloss:-1.6612 dloss:0.4991 dlossR:0.4991 dlossQ:0.2924
Episode:215 meanR:48.8200 gloss:-1.5960 dloss:0.3767 dlossR:0.3767 dlossQ:0.3087
Episode:216 meanR:49.4700 gloss:-1.6177 dloss:0.4499 dlossR:0.4499 dlossQ:0.2994
Episode:217 meanR:49.6300 gloss:-1.5549 dloss:0.2263 dlossR:0.2263 dlossQ:0.3139
Episode:218 meanR:49.7000 gloss:-1.5685 dloss:0.1847 dlossR:0.1847 dlossQ:0.3104
Episode:219 meanR:49.8300 gloss:-1.6101 dloss:0.2143 dlossR:0.2143 dlossQ:0.2989
Episode:220 meanR:49.9800 gloss:-1.5808 dloss:0.2252 dlossR:0.2252 dlossQ:0.3069
Episode:221 meanR:49.9400 gl

Episode:321 meanR:37.9400 gloss:-1.8171 dloss:0.1362 dlossR:0.1362 dlossQ:0.2469
Episode:322 meanR:37.9300 gloss:-1.8834 dloss:0.1478 dlossR:0.1478 dlossQ:0.2314
Episode:323 meanR:37.9400 gloss:-1.9400 dloss:0.1454 dlossR:0.1454 dlossQ:0.2194
Episode:324 meanR:37.9900 gloss:-1.9716 dloss:0.1555 dlossR:0.1555 dlossQ:0.2131
Episode:325 meanR:38.0100 gloss:-2.0082 dloss:0.1322 dlossR:0.1322 dlossQ:0.2062
Episode:326 meanR:38.0100 gloss:-2.0714 dloss:0.1300 dlossR:0.1300 dlossQ:0.1940
Episode:327 meanR:38.0300 gloss:-2.0977 dloss:0.1407 dlossR:0.1407 dlossQ:0.1893
Episode:328 meanR:37.9900 gloss:-2.0926 dloss:0.1523 dlossR:0.1523 dlossQ:0.1902
Episode:329 meanR:37.9500 gloss:-2.1001 dloss:0.1347 dlossR:0.1347 dlossQ:0.1892
Episode:330 meanR:38.2900 gloss:-1.9332 dloss:0.4118 dlossR:0.4118 dlossQ:0.2283
Episode:331 meanR:39.8200 gloss:-1.8965 dloss:1.0858 dlossR:1.0858 dlossQ:0.2375
Episode:332 meanR:39.7700 gloss:-2.1225 dloss:0.2576 dlossR:0.2576 dlossQ:0.1879
Episode:333 meanR:39.7100 gl

Episode:434 meanR:28.5200 gloss:-2.3795 dloss:0.1094 dlossR:0.1094 dlossQ:0.1507
Episode:435 meanR:28.3500 gloss:-2.3963 dloss:0.1086 dlossR:0.1086 dlossQ:0.1488
Episode:436 meanR:27.9600 gloss:-2.4515 dloss:0.1085 dlossR:0.1085 dlossQ:0.1446
Episode:437 meanR:27.6500 gloss:-2.5578 dloss:0.1039 dlossR:0.1039 dlossQ:0.1212
Episode:438 meanR:27.2200 gloss:-2.5790 dloss:0.1174 dlossR:0.1174 dlossQ:0.1171
Episode:439 meanR:27.2200 gloss:-2.6764 dloss:0.1034 dlossR:0.1034 dlossQ:0.1077
Episode:440 meanR:27.2500 gloss:-2.7172 dloss:0.1178 dlossR:0.1178 dlossQ:0.1022
Episode:441 meanR:27.2400 gloss:-2.7003 dloss:0.1037 dlossR:0.1037 dlossQ:0.1064
Episode:442 meanR:27.2100 gloss:-2.7002 dloss:0.0969 dlossR:0.0969 dlossQ:0.1088
Episode:443 meanR:26.9600 gloss:-2.5048 dloss:0.1372 dlossR:0.1372 dlossQ:0.1257
Episode:444 meanR:26.5200 gloss:-2.2003 dloss:0.1693 dlossR:0.1693 dlossQ:0.1722
Episode:445 meanR:26.5800 gloss:-2.0520 dloss:0.2112 dlossR:0.2112 dlossQ:0.1990
Episode:446 meanR:27.0100 gl

...

Episode:4414 meanR:117.5300 gloss:-0.6276 dloss:0.5553 dlossR:0.5553 dlossQ:0.5960
Episode:4415 meanR:116.9600 gloss:-0.3846 dloss:0.6331 dlossR:0.6331 dlossQ:0.6498
Episode:4416 meanR:117.1900 gloss:-0.0589 dloss:0.7112 dlossR:0.7112 dlossQ:0.6902
Episode:4417 meanR:119.3000 gloss:-0.2477 dloss:0.8896 dlossR:0.8896 dlossQ:0.6797
Episode:4418 meanR:118.8300 gloss:-0.1197 dloss:0.6873 dlossR:0.6873 dlossQ:0.6827
Episode:4419 meanR:118.1900 gloss:-0.3435 dloss:0.6174 dlossR:0.6174 dlossQ:0.6521
Episode:4420 meanR:118.0700 gloss:-0.3982 dloss:0.6062 dlossR:0.6062 dlossQ:0.6499
Episode:4421 meanR:116.9500 gloss:-0.9970 dloss:0.3534 dlossR:0.3534 dlossQ:0.4988
Episode:4422 meanR:116.1400 gloss:-1.6087 dloss:0.2195 dlossR:0.2195 dlossQ:0.3369
Episode:4423 meanR:115.2500 gloss:-1.6087 dloss:0.2101 dlossR:0.2101 dlossQ:0.3307
Episode:4424 meanR:115.4100 gloss:-0.5641 dloss:0.5161 dlossR:0.5161 dlossQ:0.6048
Episode:4425 meanR:115.8100 gloss:-0.3639 dloss:0.5858 dlossR:0.5858 dlossQ:0.6525
Epis

Episode:4513 meanR:111.4200 gloss:-0.4042 dloss:0.8718 dlossR:0.8718 dlossQ:0.6518
Episode:4514 meanR:111.9100 gloss:-0.3870 dloss:0.6389 dlossR:0.6389 dlossQ:0.6532
Episode:4515 meanR:111.0800 gloss:-0.4689 dloss:0.5220 dlossR:0.5220 dlossQ:0.6331
Episode:4516 meanR:109.6800 gloss:-0.4037 dloss:0.6798 dlossR:0.6798 dlossQ:0.6523
Episode:4517 meanR:106.3900 gloss:-0.4296 dloss:0.6537 dlossR:0.6537 dlossQ:0.6482
Episode:4518 meanR:105.6500 gloss:-0.3840 dloss:0.6011 dlossR:0.6011 dlossQ:0.6533
Episode:4519 meanR:106.8900 gloss:-0.2731 dloss:0.7171 dlossR:0.7171 dlossQ:0.6709
Episode:4520 meanR:108.5800 gloss:-0.3602 dloss:0.7821 dlossR:0.7821 dlossQ:0.6605
Episode:4521 meanR:111.8200 gloss:-0.4406 dloss:0.8969 dlossR:0.8969 dlossQ:0.6461
Episode:4522 meanR:113.6400 gloss:-0.4594 dloss:0.6819 dlossR:0.6819 dlossQ:0.6400
Episode:4523 meanR:115.1500 gloss:-0.4894 dloss:0.6396 dlossR:0.6396 dlossQ:0.6343
Episode:4524 meanR:115.2700 gloss:-0.7083 dloss:0.4959 dlossR:0.4959 dlossQ:0.5794
Epis

Episode:4615 meanR:121.2200 gloss:-0.2795 dloss:0.7168 dlossR:0.7168 dlossQ:0.6704
Episode:4616 meanR:121.1800 gloss:-0.4417 dloss:0.6696 dlossR:0.6696 dlossQ:0.6430
Episode:4617 meanR:120.2500 gloss:-1.0735 dloss:0.5053 dlossR:0.5053 dlossQ:0.4567
Episode:4618 meanR:119.1500 gloss:-2.3317 dloss:0.1984 dlossR:0.1984 dlossQ:0.2303
Episode:4619 meanR:116.8200 gloss:-1.9906 dloss:0.2053 dlossR:0.2053 dlossQ:0.2609
Episode:4620 meanR:114.3400 gloss:-1.2125 dloss:0.3479 dlossR:0.3479 dlossQ:0.4306
Episode:4621 meanR:112.9700 gloss:-0.4078 dloss:0.7190 dlossR:0.7190 dlossQ:0.6486
Episode:4622 meanR:115.3800 gloss:-0.2735 dloss:0.8671 dlossR:0.8671 dlossQ:0.6711
Episode:4623 meanR:114.6700 gloss:-0.4147 dloss:0.5732 dlossR:0.5732 dlossQ:0.6459
Episode:4624 meanR:114.1900 gloss:-1.1509 dloss:0.3500 dlossR:0.3500 dlossQ:0.4694
Episode:4625 meanR:113.8200 gloss:-1.0403 dloss:0.3635 dlossR:0.3635 dlossQ:0.5056
Episode:4626 meanR:114.7400 gloss:-0.4966 dloss:0.6322 dlossR:0.6322 dlossQ:0.6334
Epis

Episode:4715 meanR:164.7500 gloss:-0.2622 dloss:0.6707 dlossR:0.6707 dlossQ:0.6667
Episode:4716 meanR:163.0700 gloss:-0.8074 dloss:0.3884 dlossR:0.3884 dlossQ:0.5593
Episode:4717 meanR:165.5700 gloss:-0.2051 dloss:0.7568 dlossR:0.7568 dlossQ:0.6774
Episode:4718 meanR:166.5000 gloss:-0.8501 dloss:0.5252 dlossR:0.5252 dlossQ:0.5343
Episode:4719 meanR:166.5400 gloss:-1.3326 dloss:0.2927 dlossR:0.2927 dlossQ:0.4191
Episode:4720 meanR:166.8400 gloss:-1.0388 dloss:0.4639 dlossR:0.4639 dlossQ:0.4793
Episode:4721 meanR:164.7200 gloss:-1.2593 dloss:0.2915 dlossR:0.2915 dlossQ:0.4347
Episode:4722 meanR:161.0400 gloss:-1.1411 dloss:0.4141 dlossR:0.4141 dlossQ:0.4474
Episode:4723 meanR:160.8500 gloss:-1.2385 dloss:0.4449 dlossR:0.4449 dlossQ:0.4161
Episode:4724 meanR:160.6400 gloss:-1.5136 dloss:0.2508 dlossR:0.2508 dlossQ:0.3665
Episode:4725 meanR:160.4600 gloss:-1.4706 dloss:0.2564 dlossR:0.2564 dlossQ:0.3802
Episode:4726 meanR:159.0900 gloss:-1.9066 dloss:0.2018 dlossR:0.2018 dlossQ:0.2799
Epis

Episode:4817 meanR:160.4100 gloss:-0.7555 dloss:0.6454 dlossR:0.6454 dlossQ:0.5632
Episode:4818 meanR:161.2000 gloss:-0.7569 dloss:0.7108 dlossR:0.7108 dlossQ:0.5632
Episode:4819 meanR:162.3300 gloss:-0.9369 dloss:0.6143 dlossR:0.6143 dlossQ:0.5080
Episode:4820 meanR:162.7700 gloss:-1.1157 dloss:0.6053 dlossR:0.6053 dlossQ:0.4574
Episode:4821 meanR:162.7400 gloss:-1.7686 dloss:0.2157 dlossR:0.2157 dlossQ:0.2972
Episode:4822 meanR:162.1500 gloss:-2.0872 dloss:0.1963 dlossR:0.1963 dlossQ:0.2615
Episode:4823 meanR:162.7000 gloss:-0.8818 dloss:0.6135 dlossR:0.6135 dlossQ:0.5262
Episode:4824 meanR:165.5700 gloss:-0.3168 dloss:0.7811 dlossR:0.7811 dlossQ:0.6611
Episode:4825 meanR:167.1200 gloss:-0.2233 dloss:0.6642 dlossR:0.6642 dlossQ:0.6786
Episode:4826 meanR:167.1000 gloss:-2.0368 dloss:0.2115 dlossR:0.2115 dlossQ:0.2808
Episode:4827 meanR:165.8500 gloss:-2.3671 dloss:0.1700 dlossR:0.1700 dlossQ:0.2144
Episode:4828 meanR:166.2800 gloss:-1.3852 dloss:0.4315 dlossR:0.4315 dlossQ:0.3755
Epis

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [28]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        if total_reward == 500:
            break
                
# Closing the env
env.close()

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model2.ckpt
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.